![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
print(cc_apps.head())
print(cc_apps.shape)

ModuleNotFoundError: No module named 'sklearn'

We'll start by cleaning the data, scaling and dealing with missing values first.
## 1. Data Cleaning
###    **1.1 Dealing with missing Values**

In [ ]:
# Check raw unique values per column
for col in cc_apps.columns:
    if "?" in cc_apps[col].unique():
        print(f"Column {col} has '?'")
cc_apps = cc_apps.replace('?', np.nan) #as in UCI missing values are labeled as '?' which function won't identify as NaN
print(cc_apps.isna().sum().sort_values()) 
imputer = SimpleImputer(strategy='most_frequent')
cc_apps = pd.DataFrame(imputer.fit_transform(cc_apps), columns=cc_apps.columns)
print(cc_apps.isna().sum().sort_values()) 

Column 0 has '?'
Column 1 has '?'
Column 3 has '?'
Column 4 has '?'
Column 5 has '?'
Column 6 has '?'
2      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
3      6
4      6
5      9
6      9
0     12
1     12
dtype: int64
2          0
1_42.50    0
1_42.25    0
1_42.17    0
1_42.08    0
          ..
1_27.25    0
1_27.17    0
1_27.00    0
1_28.00    0
11_s       0
Length: 382, dtype: int64


As seen the data has no ? or NaN so it has no missing values so there is no need to deal with it.
### **1.2 Encoding categorical data**

In [ ]:
cat_cols = cc_apps.select_dtypes(include=['object']).columns
cc_apps = pd.get_dummies(cc_apps, columns=cat_cols, drop_first=True)
print(cc_apps)

          2     7   10     12  0_b  1_15.17  ...  6_v  6_z  8_t  9_t  11_p  11_s
0     0.000  1.25  1.0    0.0  1.0      0.0  ...  1.0  0.0  1.0  1.0   0.0   0.0
1     4.460  3.04  6.0  560.0  0.0      0.0  ...  0.0  0.0  1.0  1.0   0.0   0.0
2     0.500  1.50  0.0  824.0  0.0      0.0  ...  0.0  0.0  1.0  0.0   0.0   0.0
3     1.540  3.75  5.0    3.0  1.0      0.0  ...  1.0  0.0  1.0  1.0   0.0   0.0
4     5.625  1.71  0.0    0.0  1.0      0.0  ...  1.0  0.0  1.0  0.0   0.0   1.0
..      ...   ...  ...    ...  ...      ...  ...  ...  ...  ...  ...   ...   ...
685  10.085  1.25  0.0    0.0  1.0      0.0  ...  0.0  0.0  0.0  0.0   0.0   0.0
686   0.750  2.00  2.0  394.0  0.0      0.0  ...  1.0  0.0  0.0  1.0   0.0   0.0
687  13.500  2.00  1.0    1.0  0.0      0.0  ...  0.0  0.0  0.0  1.0   0.0   0.0
688   0.205  0.04  0.0  750.0  1.0      0.0  ...  1.0  0.0  0.0  0.0   0.0   0.0
689   3.375  8.29  0.0    0.0  1.0      0.0  ...  0.0  0.0  0.0  0.0   0.0   0.0

[690 rows x 382 columns]


### **1.3 Spliting the data**

In [ ]:
cc_apps.columns = cc_apps.columns.astype(str)
X = cc_apps.iloc[:,:-1]
y = cc_apps.iloc[:,-1].map({'+': 1, '-': 0})
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42, stratify = y)

### **1.4 Data Scaling**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## **2. The prediction model**

In [ ]:
param_grid = {'C': np.logspace(-4,4,20), "solver": ['liblinear','lbfgs']}
log_reg = LogisticRegression()
grid_search = GridSearchCV(log_reg, param_grid, cv = 5, scoring = 'accuracy')
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'solver': ['liblinear', 'lbfgs']},
             scoring='accuracy')

## 3. Test Model Performance 

In [ ]:
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
best_score = best_model.score(X_test_scaled, y_test)
print("Test Accuracy:", best_score)

Best Parameters:  {'C': 0.0001, 'solver': 'lbfgs'}
Best Score:  0.9171821305841924
Test Accuracy: 0.9178743961352657
